In [1]:
# import numpy as np
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# from loguru import logger as logger
# import open3d as o3d
# from tqdm import tqdm

# mesh_file = "../data/Daiwu_sample/mesh.ply"
# trajectory_file = "../data/Daiwu_sample/vo_pano.txt"


# mesh = o3d.io.read_triangle_mesh(mesh_file)
# vertices = np.asarray(mesh.vertices)
# logger.info("Vertices number in mesh.ply is {}".format(len(vertices)))

# trajectories = np.loadtxt(trajectory_file)


# # Simulated data: Replace this with your actual point cloud data
# # Example data format: [[x1, y1, z1], [x2, y2, z2], ...]
# # point_cloud_data = np.random.rand(100, 3)  # Replace with your data

# # Camera position and pose: Replace with your actual values
# # camera_position = np.array([1.0, 2.0, 3.0])
# # camera_pose = np.array([0.5, 0.5, 0.5, 0.5])  # [qw, qx, qy, qz]

# # 遍历每个关键帧
# camera_points = []
# camera_poses = []
# for idx, trajectory in tqdm(enumerate(trajectories), total=len(trajectories), desc='Processing keyframes'): # 获取相机姿态
#     timestamp = trajectory[0]
#     camera_point = trajectory[1:4]
#     qx, qy, qz, qw = trajectory[4:]
#     quaternion = [qw, qx, qy, qz]
#     camera_points.append(camera_point)
#     camera_poses.append(quaternion)


# # Visualization
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# # Plot point cloud
# ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], c='b', marker='o', label='Point Cloud')

# # Plot camera position
# ax.scatter(camera_points[:, 0], camera_points[:, 1], camera_points[:, 2], c='r', marker='s', s=100, label='Camera Position')



In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from loguru import logger as logger
import open3d as o3d
from tqdm import tqdm

#  Visualization using Open3D
def visualize_point_cloud_and_cameras(vertices, camera_poses):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(vertices)

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    cnt = 0
    for camera_pose in camera_poses:
        cnt += 1
        if (cnt == 255):
            camera = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6)
        else:
            camera = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3)
        camera.transform(camera_pose)
        vis.add_geometry(camera)

    vis.run()
    vis.destroy_window()




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
mesh_file = "../data/Daiwu_sample/mesh.ply"
trajectory_file = "../data/Daiwu_sample/vo_pano.txt"

mesh = o3d.io.read_triangle_mesh(mesh_file)
vertices = np.asarray(mesh.vertices)
logger.info("Vertices number in mesh.ply is {}".format(len(vertices)))

trajectories = np.loadtxt(trajectory_file)

# Extract camera poses
camera_poses = []
for trajectory in tqdm(trajectories, total=len(trajectories), desc='Processing trajectories'):
    x, y, z, qx, qy, qz, qw = trajectory[1:]
    quaternion = [qw, qx, qy, qz]
    # rotation_matrix = np.array([[1 - 2 * (qz**2 + qw**2), 2 * (qx * qy - qz * qw), 2 * (qx * qz + qy * qw)],
    #                             [2 * (qx * qy + qz * qw), 1 - 2 * (qx**2 + qw**2), 2 * (qy * qz - qx * qw)],
    #                             [2 * (qx * qz - qy * qw), 2 * (qy * qz + qx * qw), 1 - 2 * (qx**2 + qy**2)]])
    rotation_matrix = o3d.geometry.get_rotation_matrix_from_quaternion(quaternion) # TODO: check o3d rotation matrix input

    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation_matrix
    transformation_matrix[:3, 3] = np.array([x, y, z])
    camera_poses.append(transformation_matrix)

# Filter points based on z > 4
filtered_vertices = vertices[vertices[:, 2] < 2]

# Visualization
visualize_point_cloud_and_cameras(filtered_vertices, camera_poses)


2023-08-15 19:43:36.187 | INFO     | __main__:<module>:6 - Vertices number in mesh.ply is 9462482
Processing trajectories: 100%|██████████| 258/258 [00:00<00:00, 84806.46it/s]
